In [ ]:
import tkinter as tk
from tkinter import messagebox
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import json
import os

class PortfolioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Portföy Yönetimi")
        
        # Load portfolio data before creating the screen
        self.portfolio = self.load_portfolio()
        
        # Create the portfolio screen
        self.create_portfolio_screen()

    def create_portfolio_screen(self):
        self.portfolio_frame = tk.Frame(self.root)
        self.portfolio_frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(self.portfolio_frame, text="Portföy Yönetimi", font=("Arial", 16)).pack(pady=10)

        tk.Label(self.portfolio_frame, text="Hisse Senedi Kodu:").pack(pady=5)
        self.add_stock_entry = tk.Entry(self.portfolio_frame)
        self.add_stock_entry.pack(pady=5)

        tk.Label(self.portfolio_frame, text="Miktar:").pack(pady=5)
        self.quantity_entry = tk.Entry(self.portfolio_frame)
        self.quantity_entry.pack(pady=5)

        # Frame for buttons
        button_frame = tk.Frame(self.portfolio_frame)
        button_frame.pack(pady=10)

        self.add_button = tk.Button(button_frame, text="Ekle", command=self.add_stock)
        self.add_button.pack(side=tk.LEFT, padx=5)

        self.edit_button = tk.Button(button_frame, text="Düzenle", command=self.edit_stock)
        self.edit_button.pack(side=tk.LEFT, padx=5)

        self.portfolio_listbox = tk.Listbox(self.portfolio_frame, width=50, height=10)
        self.portfolio_listbox.pack(pady=10)
        self.portfolio_listbox.bind("<<ListboxSelect>>", self.on_stock_select)

        self.total_label = tk.Label(self.portfolio_frame, text="Toplam Değer: 0", font=("Arial", 14))
        self.total_label.pack(pady=5)

        # Placeholder for matplotlib figure
        self.fig = plt.Figure(figsize=(6, 4), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.portfolio_frame)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.pack(pady=10)

        self.update_portfolio_list()
        self.update_total_value()
        self.update_pie_chart()

    def add_stock(self):
        stock_code = self.add_stock_entry.get().strip().upper()
        try:
            quantity = int(self.quantity_entry.get().strip())
        except ValueError:
            messagebox.showerror("Hata", "Miktar geçerli bir sayı olmalıdır.")
            return

        if not stock_code or quantity <= 0:
            messagebox.showerror("Hata", "Hisse senedi kodu ve miktar geçerli olmalıdır.")
            return

        self.portfolio.append((stock_code, quantity))
        self.save_portfolio()
        self.update_portfolio_list()
        self.update_total_value()
        self.update_pie_chart()

    def update_portfolio_list(self):
        self.portfolio_listbox.delete(0, tk.END)
        for stock_code, quantity in self.portfolio:
            try:
                stock = yf.Ticker(stock_code)
                price = stock.history(period="1d")['Close'].iloc[-1]
                total = quantity * price
                self.portfolio_listbox.insert(tk.END, f"{stock_code} - {quantity} adet - Toplam: {total:.2f} TL")
            except Exception as e:
                self.portfolio_listbox.insert(tk.END, f"{stock_code} - {quantity} adet - Hata: {str(e)}")

    def update_total_value(self):
        total_value = 0
        self.stock_values = []
        self.stock_labels = []
        for stock_code, quantity in self.portfolio:
            try:
                stock = yf.Ticker(stock_code)
                price = stock.history(period="1d")['Close'].iloc[-1]
                value = quantity * price
                total_value += value
                self.stock_values.append(value)
                self.stock_labels.append(f"{stock_code} ({quantity} adet)")
            except:
                pass
        self.total_label.config(text=f"Toplam Değer: {total_value:.2f} TL")
        self.update_pie_chart()

    def update_pie_chart(self):
        self.ax.clear()
        if not self.stock_values:
            self.ax.pie([1], labels=[''], colors=['lightgrey'], startangle=90)
            self.ax.set_title("Portföy Dağılımı")
        else:
            self.ax.pie(self.stock_values, labels=self.stock_labels, autopct='%1.1f%%', startangle=90)
            self.ax.set_title("Portföy Dağılımı")
        self.canvas.draw()

    def save_portfolio(self):
        with open('portfolio.json', 'w') as f:
            json.dump(self.portfolio, f)

    def load_portfolio(self):
        if os.path.exists('portfolio.json'):
            with open('portfolio.json', 'r') as f:
                return json.load(f)
        return []

    def on_stock_select(self, event):
        try:
            selected_index = self.portfolio_listbox.curselection()[0]
            self.selected_stock_code, self.selected_quantity = self.portfolio[selected_index]
            self.add_stock_entry.delete(0, tk.END)
            self.add_stock_entry.insert(0, self.selected_stock_code)
            self.quantity_entry.delete(0, tk.END)
            self.quantity_entry.insert(0, self.selected_quantity)
        except IndexError:
            pass

    def edit_stock(self):
        if not hasattr(self, 'selected_stock_code'):
            messagebox.showerror("Hata", "Bir hisse senedi seçmelisiniz.")
            return

        new_stock_code = self.add_stock_entry.get().strip().upper()
        try:
            new_quantity = int(self.quantity_entry.get().strip())
        except ValueError:
            messagebox.showerror("Hata", "Miktar geçerli bir sayı olmalıdır.")
            return

        if not new_stock_code or new_quantity <= 0:
            messagebox.showerror("Hata", "Hisse senedi kodu ve miktar geçerli olmalıdır.")
            return

        for index, (stock_code, _) in enumerate(self.portfolio):
            if stock_code == self.selected_stock_code:
                self.portfolio[index] = (new_stock_code, new_quantity)
                break

        self.save_portfolio()
        self.update_portfolio_list()
        self.update_total_value()
        self.update_pie_chart()
        self.add_stock_entry.delete(0, tk.END)
        self.quantity_entry.delete(0, tk.END)
        delattr(self, 'selected_stock_code')

if __name__ == "__main__":
    root = tk.Tk()
    app = PortfolioApp(root)
    root.mainloop()